In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import ReLU
import torch.optim as optim
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import BatchNorm1d, Linear, Dropout


# model for 1D SMILES

In [2]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, extract_features=False):
        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        if extract_features:
            return hidden.squeeze(0) # extract feature
        hidden = hidden.squeeze(0)
        dense_outputs = self.fc(hidden)
        return dense_outputs

In [3]:
class AdvancedRNNModel(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim, num_layers=2, bidirectional=False, dropout_p=0.5):
        super(AdvancedRNNModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_p, bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout_p)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

    def forward(self, text, extract_features=False):
        embedded = self.embedding(text)
        embedded = self.dropout(embedded)  
        output, (hidden, _) = self.lstm(embedded)
        
        if extract_features:
            if self.lstm.bidirectional:
                hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
            else:
                hidden = hidden.squeeze(0)
            return hidden  

        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden.squeeze(0)
        
        hidden = self.dropout(hidden) 
        dense_outputs = self.fc(hidden)
        return dense_outputs


# model for 2D graph

In [4]:
class GCNNnodes(torch.nn.Module):
    def __init__(self, num_node_features,output_dim):
        super(GCNNnodes, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 32)
        self.fc = nn.Linear(32, output_dim)
    def forward(self, data, extract_features=False):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        
        if extract_features:
            return x
        
        x = self.fc(x)
        return x

In [5]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Dropout
from torch_geometric.nn import GCNConv, global_mean_pool

class EnhancedGCNNnodes(torch.nn.Module):
    def __init__(self, num_node_features, output_dim, dropout_rate=0.5):
        super(EnhancedGCNNnodes, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.bn1 = BatchNorm1d(16)
        self.conv2 = GCNConv(16, 32)
        self.bn2 = BatchNorm1d(32)
        self.conv3 = GCNConv(32, 64)
        self.bn3 = BatchNorm1d(64)
        self.dropout = Dropout(p=dropout_rate)
        self.fc1 = Linear(64, 32)
        self.fc2 = Linear(32, output_dim)

    def forward(self, data, extract_features=False):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = F.relu(self.bn3(self.conv3(x, edge_index)))
        x = global_mean_pool(x, batch) 

        if extract_features:
            return x  
        x = self.dropout(x)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# model for 3D graph

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import NNConv, global_mean_pool
from torch.nn import Linear, ReLU, Sequential
from torch_geometric.data import Data, DataLoader

class GraphNN3D(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, output_dim):
        super(GraphNN3D, self).__init__()
        self.conv1 = GCNConv(num_node_features, 64)
        self.conv2 = GCNConv(64, 128)
        self.fc1 = torch.nn.Linear(128 + num_edge_features, 64)  # Combine node and edge features
        self.fc2 = torch.nn.Linear(64, output_dim)

    def forward(self, data, extract_features=False):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # Graph convolution layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        # Global mean pooling
        x = global_mean_pool(x, batch)  # Aggregate features from all nodes

        # Combine with edge features
        # Here we assume edge_attr is aggregated beforehand or represent global graph properties
        
        edge_feature_aggregated = edge_attr.mean(dim=0)  # Example of simple aggregation
        x = torch.cat([x, edge_feature_aggregated.unsqueeze(0).repeat(x.size(0), 1)], dim=1)
        
        if extract_features:
            # Return the combined node and edge features
            return x

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


# model for 3D voxelization data

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Simple3DCNN(nn.Module):
    def __init__(self, num_channels, output_dim):
        super(Simple3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=num_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64 * 5 * 5 * 5, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=output_dim)

    def forward(self, x, extract_features=False):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1) 
        
        if extract_features:
            return x
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



# model for MLP

In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_dim)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ComplexMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_dim, dropout_rate=0.5):
        super(ComplexMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
